## short tutorial on running MPI4py on EC2 
#### REMINDER DONT FORGET TO TERMINATE THE CLUSTER AFTER DONE!!!!

First download linking_EC2.

In [1]:
%%bash
. ~/.bashrc
pip install  --upgrade git+https://git@github.com/JonasWallin/linkingEC2

  Cloning https://git@github.com/JonasWallin/linkingEC2 to /var/folders/ct/zpjk2t696k5bv03sqq9xrpd00000gn/T/pip-erbS_c-build
Requirement already up-to-date: boto in /Users/jonaswallin/anaconda/lib/python2.7/site-packages (from linkingEC2==0.1)
  Found existing installation: linkingEC2 0.1
    Uninstalling linkingEC2-0.1:
      Successfully uninstalled linkingEC2-0.1
  Running setup.py install for linkingEC2


 First getting the information needed to start an EC2 cluster,
 I have my information stored in ~/.starcluster/config
 
 What is needed is: 
 * [security credintals](https://sites.google.com/site/emcloudprocessing/home/setting-up-running-a-cluster-on-amazon-ec2), that is acess_key_id aws_secret_key
 * [key_for_the_region](http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-key-pairs.html)
 * and an [aws region](http://docs.aws.amazon.com/general/latest/gr/rande.html#ec2_region)
 * and a [security group](http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/using-network-security.html)
     * Note that openmpi seems to require that ALL-TCP ports are open internally see link above how to set that

In [2]:
## short tutorial on running MPI4py on EC2 

from ConfigParser import ConfigParser
config = ConfigParser()
starfigconfig_folder = "/Users/jonaswallin/.starcluster/"
config.read(starfigconfig_folder + "config")

acess_key_id     = config.get('aws info', 'aws_access_key_id'    , 0)
aws_secret_key   = config.get('aws info', 'aws_secret_access_key', 0)
aws_region_name  = config.get('aws info', 'aws_region_name'      , 0)
my_key_loc       = config.get('key mykeyABC', 'key_location',0)

Now we setup the linkingEC2 object:

In [3]:
from linkingEC2 import LinkingHandler
linker = LinkingHandler(aws_secret_access_key = aws_secret_key,
                        aws_access_key_id     = acess_key_id,
                        aws_region_name       = aws_region_name,
                        key_location          = my_key_loc,
                        key_name              = 'mykeyABC')

In [4]:
linker.start_cluster('ami-d05e75b8', 't2.micro', ['linking_EC2'],2)

waiting for instances 0  to be running: 
********

waiting for instances 1  to be running: 


collecting node001 number of processes:

CalledProcessError: Command 'ssh -i ~/.ssh/mykeyABC.rsa -o 'StrictHostKeyChecking no' ubuntu@ec2-54-209-126-166.compute-1.amazonaws.com 'cat /proc/cpuinfo | grep processor | wc -l'' returned non-zero exit status 255

Install the packages required: (this will take some time)

In [4]:
import time
PACKAGES_APT = ['python-dev', 'openmpi-bin', 'libopenmpi-dev', 'python-numpy']
PACKAGES_PIP = ['mpi4py']
time_in = time.time()
linker.apt_install(PACKAGES_APT)
linker.pip_install(PACKAGES_PIP)
time_out = time.time()
import os
os.system('say "your packages is downloaded"')
print('it took {time:.2f} sec'.format(time = time_out - time_in))

installing the packages : python-dev openmpi-bin libopenmpi-dev python-numpy
update apt-get for node001
update apt-get for node002
node001 is done
node002 is done
upgrade apt-get for node001
upgrade apt-get for node002
node001 is done
node002 is done
in import_package_to_nodes n_process = 8
installing the packages : python-pipinstalling package for node001
installing package for node002
node001 is done
node002 is done

in import_package_to_nodes n_process = 8
installing the packages : mpi4pyinstalling package for node001
installing package for node002
node001 is done
node002 is done

in import_package_to_nodes n_process = 8
it took 127.69 secinstalling package for node001
installing package for node002
node001 is done
node002 is done



Create a simple python file (that I found somewhere don't recall where, sorry)

In [5]:
%%writefile calc_pi_MPI.py
from mpi4py import MPI
import numpy as np
import random
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
mpisize = comm.Get_size()
nsamples = int(12e6/mpisize)
inside = 0
random.seed(rank)
for i in range(nsamples):
    x = random.random()
    y = random.random()
if (x*x)+(y*y)<1:
    inside += 1
mypi = (4.0 * inside)/nsamples
pi = comm.reduce(mypi, op=MPI.SUM, root=0)
if rank==0:
    print (1.0 / mpisize)*pi

Overwriting calc_pi_MPI.py


Copy the file you created to all your nodes by:

In [6]:
linker.copy_files_to_nodes( file_name = 'calc_pi_MPI.py', destination='~/' ) # deafult desnination is ~/

copying calc_pi_MPI.py to external ~/
copying files to node001 done
copying files to node002 done


On your computer in bash run next cell. Like ex:
```bash
ssh  -i ~/.ssh/MYKEY.rsa -o 'StrictHostKeyChecking no'  ubuntu@ec2-54-173-155-255.compute-1.amazonaws.com
```

In [7]:
print( linker.get_ssh_login() ) # print( linker.get_ssh_login(1) )

ssh  -i ~/.ssh/mykeyABC.rsa -o 'StrictHostKeyChecking no'  ubuntu@ec2-54-174-99-160.compute-1.amazonaws.com


Then on the amazon node run:
```bash
mpirun -hostfile nodefile -n 2 python calc_pi_MPI.py
```


##TURN OF THE CLUSTER:

In [5]:
linker.terminate_cluster()